# 生成器 generator

- 使用一个超大列表时，受到内存限制，会造成存储空间的占用浪费
- 所以，需要一种一边循环一遍计算的机制，成为生成器，generator
- 创建一个generator的方法：
    - 把列表生成式的[] 改为()即可

In [34]:
# L = [x * x for x in range(10)] 列表生成式
g = (x * x for x in range(10))
print(g)

<generator object <genexpr> at 0x000000000547FA40>


- 创建L和g的区别仅在于最外层的[]和()，L是一个list，而g是一个generator。
- list可以打印，但是generator怎么打印呢？next(g)函数

In [35]:
# next(g) 每次执行顺序打印一个,只能遍历一次
print(next(g))
print(next(g))
print('='*10)
for i in range(25):
    print(next(g))

0
1
4
9
16
25
36
49
64
81


StopIteration: 

### generator 保存的是算法，即函数
- 每次调用next()就计算下一个元素的值，直到计算到最后一个元素，没有更多元素时，抛出StopIteration的错误。
### generator的调用顺序
- 函数是顺序执行，遇到return语句或者最后一行函数语句就返回。
- generator函数在每次调用next()的时候执行，遇到yield就返回一个值，下次调用再从yield处开始。


In [58]:
# 最简单的generator
def odd():
    yield 1
    yield 3
    yield 5
print(odd())

'''
实际上是先生成了一个对象：generator_obj = odd()
然后才用：next(generator_obj)不断的调用并获得下一个返回值
执行3次yield后，已经没有yield可以执行了，如果在调用odd()就要报错
'''
print(next(odd()))
print(next(odd()))
print(next(odd()))

generator_obj = odd()
for i in range(6):
    print(next(generator_obj))
    


<generator object odd at 0x000000000547FD58>
1
1
1
1
3
5


StopIteration: 

In [64]:
'''斐波拉契数列（Fibonacci）生成器构建
除第一个和第二个数外，任意一个数都可由前两个数相加得到：
1, 1, 2, 3, 5, 8, 13, 21, 34, ...

# 注意：n, x, y = 0, 0, 1
相当于：t = (y, x + y)  # t是一个tuple
            x = t[0]
            y = t[1]

'''
# 打印前 num 位斐波拉契数列
def fib(num):
    n, x, y = 0, 0, 1
    while n < num:
        yield y
        x,y = y, (x + y)
        n += 1
    return 'done'

for i in fib(5):
    print(i)
    


1
1
2
3
5


In [67]:
'''这里发现拿不到fib()的return返回值
如果一定要拿，就要捕获StopIteration错误，返回值包含在StopIteration错误e的值velue里
'''
g = fib(6)
while True:
    try:
        x = next(g)
        print(x)
    except StopIteration as e:
        print("生成器fib()的return：",e)
        break

1
1
2
3
5
8
生成器fib()的return： done


In [1]:
# 杨辉三角定义如下：
# 递增生成max行数组，数组长度等于行数
# 每一行的数组包含一组规律数字，
# 数组左右两边元素镜像相等，基数长度，有唯一最大数

def triangles(max):
    line = [1]
    time = 0
    while time <= max:
        yield(line.copy())
        for i in range(len(line)-1, 0, -1):
            line[i] += line[i-1]
        line.append(1)
        time += 1

# 以下是测试代码       

for t in triangles(10):
    print(t)


[1]
[1, 1]
[1, 2, 1]
[1, 3, 3, 1]
[1, 4, 6, 4, 1]
[1, 5, 10, 10, 5, 1]
[1, 6, 15, 20, 15, 6, 1]
[1, 7, 21, 35, 35, 21, 7, 1]
[1, 8, 28, 56, 70, 56, 28, 8, 1]
[1, 9, 36, 84, 126, 126, 84, 36, 9, 1]
[1, 10, 45, 120, 210, 252, 210, 120, 45, 10, 1]


# 迭代器 Iterable
- 可以被for-in直接循环遍历的数据类型有两类：
    - 集合型数据：如list、tuple、dict、set、str等
    - 生成器generator:包括生成器和带yield的 generator function
    - 这些对象的统称为可迭代对象Iterable
- Iterator甚至可以表示一个无限大的数据流，例如全体自然数。而使用list是永远不可能存储全体自然数的。

In [13]:
from collections import Iterable

# 导入集合包
# 可以使用isinstance()判断一个对象是否是Iterable对象：
print(isinstance([], Iterable))
print(isinstance({}, Iterable))
print(isinstance('abc', Iterable))
print(isinstance((x for x in range(10)), Iterable))
print(isinstance(100, Iterable))

# 把list、dict、str等Iterable变成Iterator可以使用iter()函数：
print(isinstance(iter([]), Iterable))
print(isinstance(iter('abc'), Iterable))

True
True
True
True
False
True
True


- 凡是可作用于for循环的对象都是Iterable类型；
- 凡是可作用于next()函数的对象都是Iterator类型，它们表示一个惰性计算的序列；
- 集合数据类型如list、dict、str等是Iterable但不是Iterator，不过可以通过iter()函数获得一个Iterator对象。
- Python的for循环本质上就是通过不断调用next()函数实现的